In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn import Linear, Conv2d, BatchNorm1d, BatchNorm2d, PReLU, ReLU
from torch.nn import Sigmoid, Dropout2d, Dropout, AvgPool2d, MaxPool2d
from torch.nn import AdaptiveAvgPool2d, Sequential, Module, Parameter

import torchvision.models as models

from collections import namedtuple

############################################
# VGG16 + Linear(in:1000, out:128) + L2Norm
#
class MaskedFaceNetVgg16(nn.Module):
    def __init__(self):
        super(MaskedFaceNetVgg16,self).__init__()
        self.vgg16 = models.vgg16(pretrained=False)
        #self.net = self.vgg16.classifier
        
        # Linear layer to output 128bit embeddings.
        self.linear = nn.Linear(1000, 128)
        
        #for p in self.net.parameters():
        #    p.requires_grad=False

    def forward(self,x):
        #x = self.net(x)
        x = self.vgg16(x)
        x = self.linear(x)
        x = F.normalize(x, p=2, dim=1)
        return x

###############################################
# ResNet50 + Linear(in:1000, out:128) + L2Norm
#
class MaskedFaceNetResNet50(nn.Module):
    def __init__(self):
        super(MaskedFaceNetResNet50,self).__init__()
        self.net = models.resnet50(pretrained=False)
        
        # Linear layer to output 128bit embeddings.
        self.linear = nn.Linear(1000, 128)

    def forward(self,x):
        x = self.net(x)
        x = self.linear(x)
        x = F.normalize(x, p=2, dim=1)
        return x

########################################################
# ResNet50 + DropOut + Linear(in:1000, out:512) + L2Norm
#
class MaskedFaceNetResNet50V2(nn.Module):
    def __init__(self):
        super(MaskedFaceNetResNet50V2,self).__init__()
        self.backbone = models.resnet50(pretrained=False)
        
        # Remove AveragePool and FC layers
        self.backbone = nn.Sequential(*list(self.backbone.children())[:-2])
        #model.classifier = nn.Sequential(*[model.classifier[i] for i in range(4)])
        
        # embedding size 512 or 128
        output_dim = 512
        self.output_layer = Sequential(Dropout(0.6),
                                       AdaptiveAvgPool2d(output_size=(1, 1)),
                                       Flatten(),
                                       Linear(in_features=2048, out_features=output_dim, bias=True),
                                       BatchNorm1d(output_dim))

    def forward(self,x):
        x = self.backbone(x)
        x = self.output_layer(x)
        #x = F.normalize(x, p=2, dim=1)
        x = l2_norm(x)
        return x

################################################
# ResNet + Dropout + Linear + L2Norm
# Inspired by InsightFace
#
class Flatten(nn.Module):
    def forward(self, input):
        return input.view(input.size(0), -1)

def l2_norm(input,axis=1):
    norm = torch.norm(input,2,axis,True)
    output = torch.div(input, norm)
    return output

class SEModule(nn.Module):
    def __init__(self, channels, reduction):
        super(SEModule, self).__init__()
        self.avg_pool = AdaptiveAvgPool2d(1)
        self.fc1 = Conv2d(
            channels, channels // reduction, kernel_size=1, padding=0 ,bias=False)
        self.relu = ReLU(inplace=True)
        self.fc2 = Conv2d(
            channels // reduction, channels, kernel_size=1, padding=0 ,bias=False)
        self.sigmoid = Sigmoid()

    def forward(self, x):
        module_input = x
        x = self.avg_pool(x)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.sigmoid(x)
        return module_input * x

class bottleneck_IR(nn.Module):
    def __init__(self, in_channel, depth, stride):
        super(bottleneck_IR, self).__init__()
        if in_channel == depth:
            self.shortcut_layer = MaxPool2d(1, stride)
        else:
            self.shortcut_layer = Sequential(
                Conv2d(in_channel, depth, (1, 1), stride ,bias=False), BatchNorm2d(depth))
        self.res_layer = Sequential(
            BatchNorm2d(in_channel),
            Conv2d(in_channel, depth, (3, 3), (1, 1), 1 ,bias=False), PReLU(depth),
            Conv2d(depth, depth, (3, 3), stride, 1 ,bias=False), BatchNorm2d(depth))

    def forward(self, x):
        shortcut = self.shortcut_layer(x)
        res = self.res_layer(x)
        return res + shortcut

class bottleneck_IR_SE(nn.Module):
    def __init__(self, in_channel, depth, stride):
        super(bottleneck_IR_SE, self).__init__()
        if in_channel == depth:
            self.shortcut_layer = MaxPool2d(1, stride)
        else:
            self.shortcut_layer = Sequential(
                Conv2d(in_channel, depth, (1, 1), stride ,bias=False), 
                BatchNorm2d(depth))
        self.res_layer = Sequential(
            BatchNorm2d(in_channel),
            Conv2d(in_channel, depth, (3,3), (1,1),1 ,bias=False),
            PReLU(depth),
            Conv2d(depth, depth, (3,3), stride, 1 ,bias=False),
            BatchNorm2d(depth),
            SEModule(depth,16)
            )
    def forward(self,x):
        shortcut = self.shortcut_layer(x)
        res = self.res_layer(x)
        return res + shortcut

class Bottleneck(namedtuple('Block', ['in_channel', 'depth', 'stride'])):
    '''A named tuple describing a ResNet block.'''
    
def get_block(in_channel, depth, num_units, stride = 2):
  return [Bottleneck(in_channel, depth, stride)] + [Bottleneck(depth, depth, 1) for i in range(num_units-1)]

def get_blocks(num_layers):
    if num_layers == 50:
        blocks = [
            get_block(in_channel=64, depth=64, num_units = 3),
            get_block(in_channel=64, depth=128, num_units=4),
            get_block(in_channel=128, depth=256, num_units=14),
            get_block(in_channel=256, depth=512, num_units=3)
        ]
    elif num_layers == 100:
        blocks = [
            get_block(in_channel=64, depth=64, num_units=3),
            get_block(in_channel=64, depth=128, num_units=13),
            get_block(in_channel=128, depth=256, num_units=30),
            get_block(in_channel=256, depth=512, num_units=3)
        ]
    elif num_layers == 152:
        blocks = [
            get_block(in_channel=64, depth=64, num_units=3),
            get_block(in_channel=64, depth=128, num_units=8),
            get_block(in_channel=128, depth=256, num_units=36),
            get_block(in_channel=256, depth=512, num_units=3)
        ]
    return blocks

class MaskedFaceNetBackbone(Module):
    def __init__(self, num_layers, drop_ratio, mode='ir'):
        super(MaskedFaceNetBackbone, self).__init__()
        assert num_layers in [50, 100, 152], 'num_layers should be 50,100, or 152'
        assert mode in ['ir', 'ir_se'], 'mode should be ir or ir_se'
        blocks = get_blocks(num_layers)
        if mode == 'ir':
            unit_module = bottleneck_IR
        elif mode == 'ir_se':
            unit_module = bottleneck_IR_SE
        self.input_layer = Sequential(Conv2d(3, 64, (3, 3), 1, 1 ,bias=False), 
                                      BatchNorm2d(64), 
                                      PReLU(64))
        self.output_layer = Sequential(BatchNorm2d(512), 
                                       Dropout(drop_ratio),
                                       Flatten(),
                                       Linear(512 * 7 * 7, 512),
                                       BatchNorm1d(512))
        modules = []
        for block in blocks:
            for bottleneck in block:
                modules.append(
                    unit_module(bottleneck.in_channel,
                                bottleneck.depth,
                                bottleneck.stride))
        self.body = Sequential(*modules)
    
    def forward(self,x):
        x = self.input_layer(x)
        x = self.body(x)
        x = self.output_layer(x)
        return l2_norm(x)
